In [1]:
import pandas as pd
import numpy as np
import math
from random import random
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from tqdm import tqdm
import matplotlib.animation as ani
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.stattools import adfuller as ADF
from statsmodels.tsa.ar_model import AR

In [2]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
plt.rcParams["font.family"] = "Times New Roman"
plt.style.use('seaborn-whitegrid')
plt.style.use('seaborn-poster')
plt.style.use('seaborn-dark-palette')
plt.rcParams["mathtext.fontset"] = "cm"

In [3]:
# print all the columns
#pd.set_option('display.max_columns', None)
# print all the lines
#pd.set_option('display.max_rows', None)

In [4]:
def all_data():
    data = pd.read_csv('../dataset/radar_narr_combine.csv')
    # from "samplingperiod" to "ordinal.date", from radar dataset
    # from "year" to "visibility", from North American Regional Reanalysis (NARR) data
    columns = ['hgt.agl', 
           'uwind', 
           'vwind', 
           'air', 
           'pressure.sfc', 
           'relative.humidity', 
           'lon', 
           'lat', 
           'ordinal.date', 
           'omega', 
           'total.cloud.cover', 
           'visibility', 
           'albedo', 
           'acc.total.precip',
           'msl.pressure',
           'cape',
           'snow.cover']
    data.filtered = data[(data['clear'] > 0.7) & 
                         (data['uwind'] * data['uwind'] + data['vwind'] * data['vwind'] > 25)]    
    data.filtered = data.filtered[columns]
    
    return data, data.filtered

In [5]:
def map_plot(data, lat_min, lat_max, lon_min, lon_max):
    plt.figure()
    earth = Basemap(llcrnrlon=lon_min, llcrnrlat=lat_min, urcrnrlon=lon_max, urcrnrlat=lat_max)
    earth.drawcountries()
    earth.bluemarble(alpha=0.42)
    earth.drawcoastlines(color='#555566', linewidth=1)
    earth.scatter(data['lon'], data['lat'])
    plt.xlabel("longitude")
    plt.ylabel("latitude")
    plt.savefig('../figures/distribution.png', dpi=350)
    plt.show()
    
def line_plot(data, name):
    plt.figure()
    plt.plot(data.iloc[:,0], data.iloc[:,1], 'ro-', color = '#4169E1', alpha = 0.8, linewidth = 1, label = '%s'%name)
    plt.legend(loc="upper right")
    plt.xlabel('time')
    plt.ylabel('density of birds')
    plt.savefig('../figures/line_%s.jpg'%name)
    plt.show()
    
def scatter_plot(data):
    plt.figure(figsize=(14, 8))
    data.plot.scatter('lon', 'lat', c='linear_eta', colormap = 'jet')
    plt.show()

# Import dataset

In [38]:
def drop_radar(data):
    radar_drop = ['KMXX', 'KVAX', 'KPOE', 'KMBX', 
                  'KLGX', 'KHPX', 'KEOX', 'KEOX', 'KVBX']
    for radar in radar_drop:
        data = data.drop(data[data.radar_id == radar].index)
    return data    

def period():
    time = pd.DataFrame(pd.date_range(start="20070301", end="20070531", freq="D"), columns = ['samplingperiod'])
    for year in range(2008, 2018):
        dt1 = pd.DataFrame(pd.date_range(start="%i0301"%year, end="%i0531"%year, freq="D"), columns = ['samplingperiod'])
        time = pd.concat([time, dt1], axis = 0)
    time['samplingperiod'] = [x.strftime('%Y-%m-%d') for x in time['samplingperiod']]
    
    time.reset_index(drop = True, inplace = True)
    return time

In [12]:
data = pd.read_csv('../dataset/radar_narr_combine.csv')
# drop data before 2007, drop radars with lots of missing data
data = data.drop(data[data.samplingperiod < '2007-03-01'].index)
data.reset_index(drop = True, inplace = True)
data.to_csv('../dataset/radar_narr_combine1.csv')

In [8]:
data

,samplingperiod,radar_id,lon,lat,elev,hgt.agl,linear_eta,cbrt_eta,u,v,...,vwind,albedo,acc.total.precip,cape,pressure.sfc,msl.pressure,relative.humidity,snow.cover,total.cloud.cover,visibility
0,2007-03-01,KABR,-98.413056,45.455833,396.8,50,97.216600,4.598118,-5.686760,-0.663988,...,-0.763328,77.15,0.036665,0.0,95660.179688,100602.500,95.496857,1.0,100.0,1957.325928
1,2007-03-01,KABR,-98.413056,45.455833,396.8,150,103.106200,4.689159,-6.488700,0.835326,...,-0.183437,77.15,0.036665,0.0,95660.179688,100602.500,95.496857,1.0,100.0,1957.325928
2,2007-03-01,KABR,-98.413056,45.455833,396.8,250,130.795800,5.076113,-7.492540,2.422860,...,0.392794,77.15,0.036665,0.0,95660.179688,100602.500,95.496857,1.0,100.0,1957.325928
3,2007-03-01,KABR,-98.413056,45.455833,396.8,350,176.160000,5.605776,-8.781900,3.493760,...,1.742564,77.15,0.036665,0.0,95660.179688,100602.500,95.496857,1.0,100.0,1957.325928
4,2007-03-01,KABR,-98.413056,45.455833,396.8,450,117.554800,4.898692,-11.090400,4.636480,...,3.197215,77.15,0.036665,0.0,95660.179688,100602.500,95.496857,1.0,100.0,1957.325928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3943375,2017-05-31,KYUX,-114.656670,32.495278,53.0,2550,0.166340,0.549961,-6.050633,2.891500,...,4.585852,30.00,0.000000,0.0,99852.414062,100867.375,25.287952,0.0,7.5,20012.761719
3943376,2017-05-31,KYUX,-114.656670,32.495278,53.0,2650,0.441683,0.761559,-6.173967,3.219667,...,4.678981,30.00,0.000000,0.0,99852.414062,100867.375,25.287952,0.0,7.5,20012.761719
3943377,2017-05-31,KYUX,-114.656670,32.495278,53.0,2750,0.288477,0.660750,-6.310967,3.317433,...,4.772110,30.00,0.000000,0.0,99852.414062,100867.375,25.287952,0.0,7.5,20012.761719
3943378,2017-05-31,KYUX,-114.656670,32.495278,53.0,2850,0.224550,0.607814,-6.331600,3.519300,...,4.858119,30.00,0.000000,0.0,99852.414062,100867.375,25.287952,0.0,7.5,20012.761719


In [9]:
time = period()
print(time)
time = time['samplingperiod']
radars = np.unique(data['radar_id'])

     samplingperiod
0        2007-03-01
1        2007-03-02
2        2007-03-03
3        2007-03-04
4        2007-03-05
...             ...
1007     2017-05-27
1008     2017-05-28
1009     2017-05-29
1010     2017-05-30
1011     2017-05-31

[1012 rows x 1 columns]


# Density dataset

In [10]:
def density_data(data):
    columns = ['samplingperiod', 
               'radar_id', 
               'lon', 
               'lat', 
               'hgt.agl', 
               'linear_eta']
    data.density = data[columns]
    data.density['linear_eta'] = data.density['linear_eta']/11
    data.density.to_csv('../dataset/density.csv', index = False)
    return data.density

def drop_altitude(data):
    df = data.drop_duplicates(subset = ['samplingperiod', 'radar_id'], keep = 'first')
    print(df)
    for i in tqdm(range(len(df))):
        df.iloc[i, 2] = data[(data['samplingperiod'] == df.iloc[i, 0])&(data['radar_id'] == df.iloc[i, 1])][['linear_eta']].values.sum()
    
    df.reset_index(inplace = True, drop = True)
    return df

def time_series_(data, radars, time_series):
    
    time = time_series
    #col = pd.DataFrame(np.zeros((len(time),)), columns = ['0'])
    #time = pd.concat([time, col], axis = 1)
    time.set_index('samplingperiod', inplace = True, drop = False)
    for radar in tqdm(radars):
        #radar = 'KABR'
        col = data[data['radar_id']==radar][['samplingperiod', 'linear_eta']]
        col.rename(columns = {'linear_eta':'%s'%radar}, inplace = True)
        col.set_index('samplingperiod', inplace = True)
        time = pd.concat([time, col], axis = 1, join = 'outer')
        print(time)
        
    print(time) 
    time.reset_index(inplace = True, drop = True)
    return time

In [ ]:
df_density = data[['samplingperiod', 'radar_id', 'linear_eta']]
df_density = drop_altitude(df_density)
df_density.to_csv('../dataset/density_drop_alt.csv', index = True)
df_density = pd.read_csv('../dataset/density_drop_alt.csv')
df_density = time_series_(df_density, radars, time)
print(df_density)
df_denstiy.to_csv('../dataset/radar/time_series.csv', index = True)

## Fill in the blanks by AR

In [21]:
def AR_model(data): 
    #try:  
    #print(data)
    model = AR(train)
    model_fit = model.fit()
    #print(model_fit.predict(start=len(train), end=len(train)).iloc[0])
    return model_fit.predict(start=len(train), end=len(train)).iloc[0]
    #except Exception as reason:
    #    print(reason)

In [ ]:
df_density.reset_index(inplace = True, drop = True)

In [ ]:
k = 15
df_density = pd.read_csv('../dataset/time_series.csv')
df_density_time = df_density['samplingperiod']
df_density_radar = df_density.drop(['samplingperiod'], axis = 1)


for columname in df_density_radar.columns:
#columname = 'KAKQ'
    if df_density_radar[columname].count() != len(df_density_radar):
        loc = df_density_radar[columname][df_density_radar[columname].isnull().values==True].index.tolist()
        last = 0
        for index in loc:
            if index > last+k: #index%92 < k+times:
                train = df_density_radar[columname][last+1:index]
                df_density_radar[columname][[index]] = max(0, AR_model(train))
                #print(columname, index)
            else:    
                last = index

df_density = pd.concat([df_density_time, df_density_radar], axis = 1)
df_density = df_density.drop(df_density[df_density.samplingperiod < '2008-03-01'].index)
df_density.reset_index(inplace = True, drop = True)
print(df_density)

In [ ]:
print(df_density.isnull().sum())
df_density.to_csv('../dataset/time_filled_AR.csv', index = False)

## Filling in the blanks by linear regression

In [ ]:
def linear_regression(X_train, y_train, X_test):    
    try:
        lr = LinearRegression().fit(X_train, y_train) 

        #print("lr.coef_: {}".format(lr.coef_)) 
        #print("lr.intercept_: {}".format(lr.intercept_)) 

        #print("Training set score: {:.2f}".format(lr.score(X_train, y_train)))
        #print("Test set score: {:.2f}".format(lr.score(X_test, y_test)))

        return lr.predict(X_test)
    except Exception as reason:
        print(reason)
        
def X_y(data, index, k, times):
    y_train = data[index-times:index]
    x_test = data[index-k:index]
    x_train = data[index-k-times:index-times]
    x_train.reset_index(inplace = True, drop = True)
    for i in np.arange(times-1, 0, -1):
        line = data[index-k-i:index-i]
        line.reset_index(inplace = True, drop = True)
        x_train = pd.concat([x_train, line], axis = 1)
    
    return y_train, x_train.transpose(), x_test.transpose()

In [ ]:
k = 10
times = 5
df_density = pd.read_csv('../dataset/time_series.csv')
df_density_time = df_density['samplingperiod']
df_density_radar = df_density.drop(['samplingperiod'], axis = 1)


for columname in df_density_radar.columns:
    if df_density[columname].count() != len(df_density):
        loc = df_density[columname][df_density[columname].isnull().values==True].index.tolist()
        
        for index in loc:
            if index > k+times: #index%92 < k+times:
                y_train, x_train, x_test = X_y(df_density[columname], index, k, times)
                x_test = x_test.values
                x_test = x_test.reshape((1,10))
                #print(x_test.shape)
                df_density[columname][[index]] = linear_regression(x_train, y_train, x_test)
                
                
df_density = pd.concat([df_density_time, df_density_radar], axis = 1)
df_density = df_density.drop(df_density[df_density.samplingperiod < '2008-03-01'].index)
df_density.reset_index(inplace = True, drop = True)
print(df_density)

In [ ]:
print(df_density.isnull().sum())
df_density.to_csv('../dataset/time_filled_regression.csv', index = False)

# Weather information

In [13]:
data = pd.read_csv('../dataset/radar_narr_combine1.csv')

In [34]:
col_w = ['samplingperiod',
         'radar_id','linear_eta',
         'hgt.agl',
         'uwind',
         'vwind',
         'clear',
         'weather',
         'air',
         'omega',
         'albedo',
         'cape',
         'pressure.sfc',
         'relative.humidity', 
         'total.cloud.cover', 
         'visibility', 
         'acc.total.precip', 
         'msl.pressure', 
         'snow.cover']
data_w = data[col_w]
data_w.reset_index(drop = True, inplace = True)
info_w = data_w.set_index(['samplingperiod'], inplace = False)
info_w.to_csv('../dataset/data_w.csv', index = True)
info_w

,radar_id,linear_eta,hgt.agl,uwind,vwind,clear,weather,air,omega,albedo,cape,pressure.sfc,relative.humidity,total.cloud.cover,visibility,acc.total.precip,msl.pressure,snow.cover
samplingperiod,,,,,,,,,,,,,,,,,,
2007-03-01,KABR,97.216600,50,-4.487842,-0.763328,0.049200,0.950800,-2.879694,-0.140705,77.15,0.0,95660.179688,95.496857,100.0,1957.325928,0.036665,100602.500,1.0
2007-03-01,KABR,103.106200,150,-5.359036,-0.183437,0.049200,0.950800,-3.438355,-0.142758,77.15,0.0,95660.179688,95.496857,100.0,1957.325928,0.036665,100602.500,1.0
2007-03-01,KABR,130.795800,250,-6.176833,0.392794,0.049200,0.950800,-4.007247,-0.145240,77.15,0.0,95660.179688,95.496857,100.0,1957.325928,0.036665,100602.500,1.0
2007-03-01,KABR,176.160000,350,-7.106315,1.742564,0.049200,0.950800,-4.640103,-0.174206,77.15,0.0,95660.179688,95.496857,100.0,1957.325928,0.036665,100602.500,1.0
2007-03-01,KABR,117.554800,450,-8.050941,3.197215,0.049200,0.950800,-5.281631,-0.206762,77.15,0.0,95660.179688,95.496857,100.0,1957.325928,0.036665,100602.500,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-05-31,KYUX,0.166340,2550,-3.833800,4.585852,0.995333,0.004667,13.204676,-0.129589,30.00,0.0,99852.414062,25.287952,7.5,20012.761719,0.000000,100867.375,0.0
2017-05-31,KYUX,0.441683,2650,-3.717864,4.678981,0.995333,0.004667,12.565155,-0.136911,30.00,0.0,99852.414062,25.287952,7.5,20012.761719,0.000000,100867.375,0.0
2017-05-31,KYUX,0.288477,2750,-3.601929,4.772110,0.995333,0.004667,11.925634,-0.144234,30.00,0.0,99852.414062,25.287952,7.5,20012.761719,0.000000,100867.375,0.0


In [35]:
data_w = pd.read_csv('../dataset/data_w.csv') 
radars = np.unique(data_w['radar_id'])
time = period()
time_radar = pd.DataFrame()
hgt = pd.DataFrame(np.ones((len(time),1)), columns = ['hgt'])
for i in range(30):
    time_hgt = pd.concat([time, hgt*100*i+50], axis = 1)
    time_radar = pd.concat([time_radar, time_hgt], axis = 0)

time_radar.sort_values(['samplingperiod', 'hgt'], inplace = True)
time_radar.reset_index(inplace = True, drop = True)
#print(time_radar)
time_all = time_radar
for i in range(len(radars)-1):
    time_all = pd.concat([time_all, time_radar], axis = 0)

time_all.set_index(['samplingperiod'], inplace = True, drop = True)
print(time_all)
time_all.to_csv('../dataset/time_.csv', index = True)

                   hgt
samplingperiod        
2007-03-01        50.0
2007-03-01       150.0
2007-03-01       250.0
2007-03-01       350.0
2007-03-01       450.0
...                ...
2017-05-31      2550.0
2017-05-31      2650.0
2017-05-31      2750.0
2017-05-31      2850.0
2017-05-31      2950.0

[4341480 rows x 1 columns]


In [36]:
time_all = pd.read_csv('../dataset/time_.csv')
data_w = pd.read_csv('../dataset/data_w.csv') 
time_all.set_index('samplingperiod', inplace = True)
info_w = data_w.set_index('samplingperiod')

In [37]:
i = 0
l = int(len(time_all)/len(radars))
time_w = pd.DataFrame()

#radar = 'KABR'
for radar in tqdm(radars):
    tmp1 = pd.DataFrame()
    for h in range(50, 3000, 100):

        time_rh = time_all.iloc[i:i+l]
        #print(time_rh[time_rh['hgt'] == h])
        #print(info_w[(info_w['radar_id']==radar)&(info_w['hgt.agl']==h)])
        tmp = pd.concat([time_rh[time_rh['hgt'] == h], info_w[(info_w['radar_id']==radar)&(info_w['hgt.agl']==h)]], axis = 1, join = 'outer')
        tmp1 = pd.concat([tmp1, tmp], axis = 0)
        
    loc = tmp1['radar_id'][tmp1['radar_id'].isnull().values==True].index.tolist()   

    for index in loc:
        tmp1['radar_id'][[index]] = radar
    time_w = pd.concat([time_w, tmp1], axis = 0)
    i = i+l

time_w.reset_index(inplace = True, drop = False)
time_w.sort_values(by = ['radar_id', 'samplingperiod', 'hgt'], inplace = True)
time_w = time_w.drop('hgt.agl', axis=1)
time_w.to_csv('../dataset/time_weather.csv', index = False)
print(time_w.isnull().sum())

  0%|          | 0/143 [00:00<?, ?it/s]<ipython-input-37-2332e9010415>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp1['radar_id'][[index]] = radar
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
  1%|          | 1/143 [00:06<15:45,  6.66s/it]<ipython-input-37-2332e9010415>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

  8%|▊         | 12/143 [01:23<15:39,  7.17s/it]<ipython-input-37-2332e9010415>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp1['radar_id'][[index]] = radar
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
  9%|▉         | 13/143 [01:30<15:30,  7.16s/it]<ipython-input-37-2332e9010415>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

 17%|█▋        | 24/143 [02:47<14:17,  7.20s/it]<ipython-input-37-2332e9010415>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp1['radar_id'][[index]] = radar
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
 17%|█▋        | 25/143 [02:55<14:32,  7.39s/it]<ipython-input-37-2332e9010415>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

 25%|██▌       | 36/143 [04:23<13:55,  7.81s/it]<ipython-input-37-2332e9010415>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp1['radar_id'][[index]] = radar
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
 26%|██▌       | 37/143 [04:31<13:31,  7.66s/it]<ipython-input-37-2332e9010415>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

 34%|███▎      | 48/143 [07:01<25:27, 16.08s/it]<ipython-input-37-2332e9010415>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp1['radar_id'][[index]] = radar
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
 34%|███▍      | 49/143 [07:11<22:17, 14.23s/it]<ipython-input-37-2332e9010415>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

...

 67%|██████▋   | 96/143 [14:34<10:23, 13.27s/it]<ipython-input-37-2332e9010415>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp1['radar_id'][[index]] = radar
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
 68%|██████▊   | 97/143 [14:42<08:53, 11.60s/it]<ipython-input-37-2332e9010415>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

 76%|███████▌  | 108/143 [16:37<08:01, 13.77s/it]<ipython-input-37-2332e9010415>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp1['radar_id'][[index]] = radar
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
 76%|███████▌  | 109/143 [16:45<06:45, 11.92s/it]<ipython-input-37-2332e9010415>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

 84%|████████▍ | 120/143 [18:42<04:27, 11.61s/it]<ipython-input-37-2332e9010415>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp1['radar_id'][[index]] = radar
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
 85%|████████▍ | 121/143 [18:50<03:54, 10.64s/it]<ipython-input-37-2332e9010415>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

 92%|█████████▏| 132/143 [20:20<01:31,  8.34s/it]<ipython-input-37-2332e9010415>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp1['radar_id'][[index]] = radar
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
 93%|█████████▎| 133/143 [20:28<01:21,  8.16s/it]<ipython-input-37-2332e9010415>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

samplingperiod            0
hgt                       0
radar_id                  0
linear_eta           267154
uwind                266970
vwind                266970
clear                266970
weather              266970
air                  266970
omega                266970
albedo               266970
cape                 266970
pressure.sfc         266970
relative.humidity    266970
total.cloud.cover    266970
visibility           266970
acc.total.precip     266970
msl.pressure         266970
snow.cover           266970
dtype: int64


## Fill in the blanks of weather information

In [39]:
time_w = pd.read_csv('../dataset/time_weather.csv')
time_w = drop_radar(time_w)
print(time_w.columns)
radars = np.unique(time_w['radar_id'])

Index(['samplingperiod', 'hgt', 'radar_id', 'linear_eta', 'uwind', 'vwind',
       'clear', 'weather', 'air', 'omega', 'albedo', 'cape', 'pressure.sfc',
       'relative.humidity', 'total.cloud.cover', 'visibility',
       'acc.total.precip', 'msl.pressure', 'snow.cover'],
      dtype='object')


In [40]:
pd.DataFrame(radars).to_csv('../dataset/radar.csv', index = False)
print(radars)

['KABR' 'KABX' 'KAKQ' 'KAMA' 'KAMX' 'KAPX' 'KARX' 'KATX' 'KBBX' 'KBGM'
 'KBHX' 'KBIS' 'KBLX' 'KBMX' 'KBOX' 'KBRO' 'KBUF' 'KBYX' 'KCAE' 'KCBW'
 'KCBX' 'KCCX' 'KCLE' 'KCLX' 'KCRP' 'KCXX' 'KCYS' 'KDAX' 'KDDC' 'KDFX'
 'KDGX' 'KDIX' 'KDLH' 'KDMX' 'KDOX' 'KDTX' 'KDVN' 'KDYX' 'KEAX' 'KEMX'
 'KENX' 'KEPZ' 'KESX' 'KEVX' 'KEWX' 'KEYX' 'KFCX' 'KFDR' 'KFDX' 'KFFC'
 'KFSD' 'KFSX' 'KFTG' 'KFWS' 'KGGW' 'KGJX' 'KGLD' 'KGRB' 'KGRK' 'KGRR'
 'KGSP' 'KGWX' 'KGYX' 'KHDX' 'KHGX' 'KHNX' 'KHTX' 'KICT' 'KICX' 'KILN'
 'KILX' 'KIND' 'KINX' 'KIWA' 'KIWX' 'KJAX' 'KJGX' 'KJKL' 'KLBB' 'KLCH'
 'KLIX' 'KLNX' 'KLOT' 'KLRX' 'KLSX' 'KLTX' 'KLVX' 'KLWX' 'KLZK' 'KMAF'
 'KMAX' 'KMHX' 'KMKX' 'KMLB' 'KMOB' 'KMPX' 'KMQT' 'KMRX' 'KMSX' 'KMTX'
 'KMUX' 'KMVX' 'KNKX' 'KNQA' 'KOAX' 'KOHX' 'KOKX' 'KOTX' 'KPAH' 'KPBZ'
 'KPDT' 'KPUX' 'KRAX' 'KRGX' 'KRIW' 'KRLX' 'KRTX' 'KSFX' 'KSGF' 'KSHV'
 'KSJT' 'KSOX' 'KSRX' 'KTBW' 'KTFX' 'KTLH' 'KTLX' 'KTWX' 'KTYX' 'KUDX'
 'KUEX' 'KVNX' 'KVTX' 'KVWX' 'KYUX']


In [29]:
k = 450
radars = ['KRIW']
c1 = time_w.columns.drop(['samplingperiod', 'hgt', 'radar_id'])
for radar in tqdm(radars):
    #try:
    tmp = time_w[time_w['radar_id']==radar]
    #tmp.to_csv('../dataset/weather_filled/time_weather_filled_%s.csv'%radar, index = False)
    for col in c1:
        print(col)
        loc = tmp[col][tmp[col].isnull().values==True].index.tolist()
        start = tmp.index[0]
        last = tmp.index[0]
        for index in loc:

            if index > last+k: 
                train = tmp[last-start:index-start][col]
                v = max(min(time_w[col]), AR_model(train))
                time_w[col][[index]] = v
                tmp[col][[index]] = v

            else:    
                last = index + 1

    tmp = tmp.drop(tmp[tmp.samplingperiod < '2008-03-01'].index)
    tmp.to_csv('../dataset/weather_filled/time_weather_filled_%s.csv'%radar, index = False)


  0%|          | 0/1 [00:00<?, ?it/s]

linear_eta
uwind
vwind
clear


/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'
<ipython-input-29-383108c0ff13>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_w[col][[index]] = v
<ipython-input-29-383108c0ff13>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.p

/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  w

/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  w

...

/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  w

/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  w

/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  w

/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  w

100%|██████████| 1/1 [34:15<00:00, 2055.58s/it]


In [ ]:
k = 450
c1 = ['albedo', 'snow.cover']
for radar in tqdm(radars):
    #try:
    tmp = time_w[time_w['radar_id']==radar]
    for col in c1:
        print(col)
        loc = tmp[col][tmp[col].isnull().values==True].index.tolist()
        start = tmp.index[0]
        last = tmp.index[0]
        for index in loc:

            if index > last+k: 
                train = tmp[last-start:index-start][col]
                if train.nunique(dropna=False) == 1:
                    time_w[col][[index]] = time_w[col][[index-1]]
                    tmp[col][[index]] = tmp[col][[index-1]]
                else: 
                    v = max(min(time_w[col]), AR_model(train))
                    time_w[col][[index]] = v
                    tmp[col][[index]] = v

            else:    
                last = index + 1
                
    tmp = tmp.drop(tmp[tmp.samplingperiod < '2008-03-01'].index)
    tmp1 = pd.read_csv('../dataset/weather_filled/time_weather_filled_%s.csv'%radar)
    tmp1[col] = tmp[col]
    tmp.to_csv('../dataset/weather_filled1/time_weather_filled_%s.csv'%radar, index = False)

  0%|          | 0/135 [00:00<?, ?it/s]

albedo
snow.cover


  1%|          | 1/135 [00:02<04:52,  2.18s/it]

albedo


<ipython-input-42-8bb5c922c754>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_w[col][[index]] = time_w[col][[index-1]]
<ipython-input-42-8bb5c922c754>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp[col][[index]] = tmp[col][[index-1]]
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported 

...

/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn(

/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn(

/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  w

/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  w

/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  warnings.warn('No supported index is available.'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:578: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  warnings.warn('An unsupported index was provided and will be'
/Users/isabelle/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:376: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  w

# Correlation

In [ ]:
#correlation.pearson = data.filtered.corr(method = "pearson")
def correlation_pearson(data, columns):
    correlation = data.corr(method = "pearson")
    plt.figure()
    sns.heatmap(data.corr(method = "pearson"), xticklabels = data.corr(method = "pearson").columns, 
                yticklabels = data.corr(method = "pearson").columns, cmap = 'RdYlGn', center = 0, annot = True)
    plt.title('Correlation pearson')
    plt.xticks(np.arange(len(columns))+0.5, columns) 
    plt.yticks(np.arange(len(columns))+0.5, columns) 
    plt.savefig('../figures/correlation_pearson.png',dpi=300)
    plt.show()
    return correlation

def correlation_spearman(data, columns):
    correlation = data.corr(method = "spearman")
    plt.figure()
    sns.heatmap(data.corr(method = "spearman"), xticklabels = data.corr(method = "spearman").columns, 
                yticklabels = data.corr(method = "spearman").columns, cmap = 'RdYlGn', center = 0, annot = True)
    plt.title('Correlation Spearman')
    plt.xticks(np.arange(len(columns))+0.5, columns) 
    plt.yticks(np.arange(len(columns))+0.5, columns) 
    plt.savefig('../figures/correlation_spearman.png',dpi=300)
    plt.show()
    return correlation

In [ ]:
data, data.filtered = all_data()
columns = data.columns
cor_pearson = correlation_pearson(data, columns)
cor_spearman = correlation_spearman(data, columns)

# Plot the location of radars

In [ ]:
lat_min = min(data['lat'])
lat_max = max(data['lat'])
lon_min = min(data['lon'])
lon_max = max(data['lon'])
eta_min = min(data['linear_eta'])
eta_max = max(data['linear_eta'])
print(lat_min, lat_max, lon_min, lon_max, eta_min, eta_max)


In [ ]:
#scatter_plot(data.drop_altitude)
map_plot(data.drop_altitude, lat_min-2, lat_max+2, lon_min-2, lon_max+2)